# Geospatial Python

This notebook has a few reminders of basic python before we dig into geospatial libraries

Datasets for this binder notebook are in the relative path: /data

In [ ]:
print("Welcome to Geospatial Python!")

#### Commenting

When we write code, it is essential to add comments to help future readers of the code understand what it does

We can also "comment out" certain lines of code while we're testing! Keyboard shortcut to toggle: cntrl /

In python, the \# character is used for this... try it out!

In [ ]:
x = "purple"   # This line assigns the string "purple" to the variable x
# x = "magenta"  # This line assigns the string "magenta" to the variable x
print(x)

#### Indentation
Python uses space to separate code blocks.

In [ ]:
# The code below is an example of a conditional statement
# The indentations are important! An IDE will automatically insert them.

x = 4
if x == 3:   # the == is testing the equality of x and 3
    print("We know x is 3")
else:
    print("We know x is NOT 3")

#### Variables
Python is object-oriented so you can define strings and numbers with syntax. You don't have to declare variables! We are working with objects that hold a reference to the value you assign them.

In [ ]:
myint = 4
print(myint)

myfloat1 = 4.0
print(myfloat1)
myfloat2 = float(3)
print(myfloat2)

mystring1 = 'red'
mystring2 = "green"
# mystring3 = "Diane's favorite food is green chile"   # double quotes are good when you need an apostrophe
print(mystring1)

In [ ]:
# We can check the status of a variable by using 'type'
type(mystring1)

#### Operators
Numbers

In [ ]:
# Adding and subtracting numbers

8+3
# 8-3
# answer = myfloat1 - myfloat2
# print(answer)

In [ ]:
# Multiplication and powers

8*3
# 8**3
# (8.3)*(1.5)

In [ ]:
# Division - floating point

8/3
# Division - integer
#8//3
# Modulus - returns the remainder
#8%5

In [ ]:
# Order of operations conducted in python is PEMDAS:
# Parentheses, Exponents, Multiplication and Division (from left to right), Addition and Subtraction (from left to right)
# What we learned in elementary school!

8*3 + 4*(1+2)

Strings

In [ ]:
# Adding strings (concatenation)
# This trick is useful for writing output files!

yummy = mystring1 + " & " + mystring2 + " chile"
print(yummy)

#### Looping

Using loops to step through repetitive tasks is POWERFUL

In [ ]:
# An example of a for loop:

mylist = ["red","green","blue"]
for x in mylist:
    print(x)

#### Functions

Many are built in.

You can also define your own.

(*Libraries* are collections of functions other people have written for you!)

In [ ]:
# In Python3, "print" is now a function called with parenthesis (this wasn't true in Python2)

mysentence = "to print this sentence, I need to call the print function for the mysentence variable"
print(mysentence)



In [ ]:
# Another function is called type() and it will return the data type of your variable as we saw above

type(myint)
# type(myfloat1)
# type(mystring1)

In [ ]:
# Defining a simple function

x = 4
y = 3

def myfunction():
    result = x + y
    print(result)
    
myfunction()

---

Now, let's get into some geospatial fun!

These next sections on Geocoding and Interactive Maps are from [Melanie Walsh's Mapping chapter](https://melaniewalsh.github.io/Intro-Cultural-Analytics/welcome.html):

## GeoPy for Geocoding

Geocoding = using reference spatial data to find lat and lon points to assign to addresses

Using Nominatum and OpenStreetMap (OSM is the reference data)

GeoPy is a python library that makes this possible!

In [ ]:
# Before we can use libraries or their subfunctions, we need to import them!

from geopy.geocoders import Nominatim

In [ ]:
geolocator = Nominatim(user_agent="YOUR NAME's mapping app", timeout=2)

In [ ]:
location = geolocator.geocode("South Cayuga Street")

In [ ]:
location

In [ ]:
print(location.address)

In [ ]:
print(location.latitude, location.longitude)

In [ ]:
print(f"Importance: {location.raw['importance']}")

In [ ]:
print(f"Class: {location.raw['class']} \nType: {location.raw['type']}")

In [ ]:
possible_locations = geolocator.geocode("College Ave", exactly_one=False)

for location in possible_locations:
    print(location.address)
    print(location.latitude, location.longitude)
    print(f"Importance: {location.raw['importance']}")

In [ ]:
location = geolocator.geocode("College Ave, Ithaca NY")

print(location.address)
print(location.latitude, location.longitude)
print(f"Importance: {location.raw['importance']}")

In [ ]:
import pandas as pd
pd.set_option("max_rows", 400)
pd.set_option("max_colwidth", 400)

In [ ]:
def find_location(row):
    
    place = row['place']
    
    location = geolocator.geocode(place)
    
    if location != None:
        return location.address, location.latitude, location.longitude, location.raw['importance']
    else:
        return "Not Found", "Not Found", "Not Found", "Not Found"

In [ ]:
ithaca_df = pd.read_csv("data/IthacaPlaces.csv")

In [ ]:
ithaca_df[['address', 'lat', 'lon', 'importance']] = ithaca_df.apply(find_location, axis="columns", result_type="expand")
ithaca_df

## Folium for Making Interactive Maps

To map our geocoded coordinates, we’re going to use the Python library Folium. Folium is built on top of the popular JavaScript library Leaflet.

In [ ]:
import folium

In [ ]:
ithaca_map = folium.Map(location=[42.44, -76.5], zoom_start=14)
ithaca_map

#### Add a Marker

In [ ]:
folium.Marker(location=[42.451851, -76.477850], popup="Olde Tyme Ice Skating").add_to(ithaca_map)
ithaca_map

#### Add Markers from Pandas Dataframe

To add markers for every location in our Pandas dataframe, we can make a Python function and .apply() it to every row in the dataframe.

In [ ]:
def create_map_markers(row, map_name):
    folium.Marker(location=[row['lat'], row['lon']], popup=row['place']).add_to(map_name)

In [ ]:
found_ithaca_locations = ithaca_df[ithaca_df['address'] != "Not Found"]

In [ ]:
found_ithaca_locations.apply(create_map_markers, map_name=ithaca_map, axis='columns')
ithaca_map

#### Save Map

In [ ]:
ithaca_map.save("Ithaca-map.html")

---

## Custom Map Backgrounds

Using folium, we can pull in all sorts of map tiles for our basemaps.

There is a whole library of [leaflet providers](https://leaflet-extras.github.io/leaflet-providers/preview/) available... just copy and paste the tiles and attributes information for the tiles you want to use!

In [ ]:
import folium

In [ ]:
folium.Map(location=[0, 30],
           zoom_start=2,
           tiles='http://c.tile.stamen.com/watercolor/{z}/{x}/{y}.jpg',
           attr='Map tiles by <a href="http://stamen.com">Stamen Design</a>, under <a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a>. Data by <a href="http://openstreetmap.org">OpenStreetMap</a>, under <a href="http://creativecommons.org/licenses/by-sa/3.0">CC BY SA</a>.')

In [ ]:
# For the shipwrecks we'll investigate below, an ocean-centric basemap is appropriate!

folium.Map(location=[40, -80],
           zoom_start= 3,
           tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Ocean_Basemap/MapServer/tile/{z}/{y}/{x}',
           attr='Tiles &copy; Esri &mdash; Sources: GEBCO, NOAA, CHS, OSU, UNH, CSUMB, National Geographic, DeLorme, NAVTEQ, and Esri')

## Pandas and GeoPandas for Geospatial Data

We often think of geospatial data in the form of points, lines and polygons.

Points are great for individual markers, but what if we want to bring in a road network or ecoregions onto our map?

We'll find these point, line, or polygon geospatial datasets in open data portals in different file formats like shapefiles and geojsons. Geopandas can easily read all these vector-based data!

In [ ]:
# pandas is a useful library for importing and working with spreadsheets (aka dataframes)
# geopandas is a library built on pandas that is geospatially enabled
# you'll also need a plotting library like matplotlib if you're not using leaflet maps in folium

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
# first, we'll read in a .csv with pandas
# Our first sample dataset is of earthquakes recorded up to the year 2000 and limited to magnitudes >= 6

quakes = pd.read_csv("data/quakes2000_Mag6above.csv")
quakes.head(3)   # head will return the top lines of your dataframe that you choose

Notice two of the columns in our .csv are lat and lon information! We can pull any spreadsheet into pandas that has this information and then plot it on a map with geopandas.

In [ ]:
# Now we'll use geopandas to geoenable our quakes with the Lat and Lon columns

geoquakes = gpd.GeoDataFrame(quakes, geometry=gpd.points_from_xy(quakes.Lon, quakes.Lat))

geoquakes.head(3)   # Now we'll see a new column called "geometry"

In [ ]:
# We can use this geometry column to plot the data on a map!

geoquakes.plot()  # Here, plot() is a built-in function within the geopandas library

In [ ]:
# Geopandas can directly read shapefiles or other vector formats!
# Let's plot the earthquakes over the natural earth land shapefile of the world so we can see where they are.
# We can also change the styly of our markers

world = gpd.read_file("data/ne_110m_land/ne_110m_land.shp")
basemap = world.plot(color = 'grey')  
geoquakes.plot(ax=basemap, marker='*', color='red', markersize=30)
plt.show()

In [ ]:
# Now let's look at a different dataset about shipwrecks
# Datasource: https://nauticalcharts.noaa.gov/data/wrecks-and-obstructions.html

wrecks = pd.read_csv("data/AWOIS_Wrecks.csv")
wrecks.head()   # head will return the top lines of your dataframe - here, leaving it blank will return 5 rows as the default

In [ ]:
# Now we'll use geopandas to geoenable our quakes with the Lat and Lon columns

geowrecks = gpd.GeoDataFrame(wrecks, geometry=gpd.points_from_xy(wrecks.LONDEC, wrecks.LATDEC))

geowrecks.head(3)   # Now we'll see a new column called "geometry"

In [ ]:
yearsunkwrecks = geowrecks.dropna(subset=['YEARSUNK'])
yearsunkwrecks

In [ ]:
world = gpd.read_file("data/ne_110m_land/ne_110m_land.shp")
basemap = world.plot(color='grey')
yearsunkwrecks.plot(ax=basemap, marker='h', markersize=100, column='YEARSUNK', cmap='Greens', legend=True)
basemap.set_xlim(-170, -50)
basemap.set_ylim(10, 80)
plt.show()

In [ ]:
# Let's add a rivers layer to our map (not very pretty, but you get the idea)

world = gpd.read_file("data/ne_110m_land/ne_110m_land.shp")
basemap = world.plot(color='grey')
rivers = gpd.read_file("data/ne_50m_rivers_lake_centerlines.shp")
rivers.plot(ax=basemap, color='blue', zorder=1)
yearsunkwrecks.plot(ax=basemap, marker='h', markersize=100, column='YEARSUNK', cmap='Greens', legend=True, zorder=2)
basemap.set_xlim(-170, -50)
basemap.set_ylim(10, 80)
plt.show()


## Coordinate Reference Systems

For our datasets to line up for analysis, we need to make sure they all have the same crs

In [ ]:
# We can easily check the crs of our land basemap with the following command:

world.crs

In [ ]:
# Now instead of starting with the AWOIS shipwrecks spreadsheet, let's pull in the shapefile

world = gpd.read_file("data/ne_110m_land.shp")
basemap = world.plot()
AWOISshp = gpd.read_file("data/AWOIS_Wrecks.shp")
AWOISshp.plot(ax=basemap, color='red', markersize=3)
basemap.set_xlim(-170, -50)
basemap.set_ylim(10, 80)
plt.show()

Notice something different about how North America looks compared to the map we had earlier?

In [ ]:
# Let's check the crs of our AWOIS layer:

AWOISshp.crs

It's different! The easiest way to know is to refer to the EPSG codes... you can find more about each one at https://epsg.io/

In [ ]:
# Changing the crs of a layer can be done with the following:

world_NAD83 = world.to_crs(epsg=4269)